In [15]:
import csv
import io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
import nltk
nltk.download('stopwords')

import pandas as pd
import numpy as np
import joblib
from time import time
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
%matplotlib inline

print(tf.__version__)

#urls = 'https://github.com/dragon-library/work_space/raw/main/HS_Code/HS/hs_code.xlsx'

types = 'section'


def get_master(urls,sheets,types = 'section'):
    data = pd.read_excel(urls,sheet_name= sheets)
    data[types] = data[types].map('{:02}'.format)
    data = data[[types,'description']]
    data['description'] = data['description'].str.lower()
 #   data = data.rename(columns={'heading' : 'target', 'product_desc' : 'question_text'})
     

    return data

def manage_data(df):
    df.columns = ['target', 'description']       
    
    return df

2.1.0


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
%%time

urls = 'data/random_test.xlsx'
types = "section"

sheets = 'Sheet1'
data = get_master(urls,sheets,types)
#data.dropna(inplace=True)
decl = data[0:20000]


XLRDError: Excel xlsx file; not supported

In [12]:
%%time
urls = 'data/hs_code.xlsx'
types = "section"

sheets = '8_digit'
inputs_01 = get_master(urls,sheets,types)
sheets = '2_digit'
inputs_02 = get_master(urls,sheets,types)
sheets = '6_digit'
inputs_03 = get_master(urls,sheets,types)
sheets = '4_digit'
inputs_04 = get_master(urls,sheets,types)

data = pd.concat([inputs_01,inputs_02,inputs_03,inputs_04], ignore_index=True)
df = manage_data(data)

print(len(df.description))
df.head()

XLRDError: Excel xlsx file; not supported

In [7]:
vocab_size = 5000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

labels = df.target
articles = df.description

train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_articles))
print(len(train_labels))
print(len(validation_articles))
print(len(validation_labels))

NameError: name 'df' is not defined

In [8]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'or': 2,
 'of': 3,
 'and': 4,
 'other': 5,
 'not': 6,
 'in': 7,
 'for': 8,
 'than': 9,
 'no': 10}

In [9]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
print(train_sequences[10])

[705, 470, 175, 5, 9, 3739, 4, 1776, 2368, 1109, 470]


In [10]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

8
100
8
100
11
100


In [11]:
print(train_padded[10])

[ 705  470  175    5    9 3739    4 1776 2368 1109  470    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [12]:
#train_sequences = tokenizer.texts_to_sequences(train_articles)
#train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

7444
(7444, 100)


In [13]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[11]
[11]
[11]
(29775, 1)
[4]
[3]
[1]
(7444, 1)


In [14]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(train_padded[10]))
print('---')
print(train_articles[10])

bovine animals live other than cattle and buffalo purebred breeding animals ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
---
bovine animals; live, other than cattle and buffalo - purebred breeding animals


In [15]:
def load_model(paths=None):
    loaded_keras_model = tf.keras.models.load_model(paths)

    return loaded_keras_model


def save_model(models,paths = None):
    keras_model_path = paths
    models.save(keras_model_path)
    print("save model success")

In [16]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(25, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1625      
Total params: 395,929
Trainable params: 395,929
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/20
931/931 - 55s - loss: 1.1410 - accuracy: 0.6674 - val_loss: 1.0249 - val_accuracy: 0.6985
Epoch 2/20
931/931 - 51s - loss: 0.4116 - accuracy: 0.8803 - val_loss: 0.9131 - val_accuracy: 0.7319
Epoch 3/20
931/931 - 52s - loss: 0.3150 - accuracy: 0.9017 - val_loss: 0.9162 - val_accuracy: 0.7364
Epoch 4/20
931/931 - 51s - loss: 0.2681 - accuracy: 0.9147 - val_loss: 0.8963 - val_accuracy: 0.7429
Epoch 5/20
931/931 - 54s - loss: 0.2432 - accuracy: 0.9209 - val_loss: 0.9367 - val_accuracy: 0.7444
Epoch 6/20
931/931 - 52s - loss: 0.2194 - accuracy: 0.9274 - val_loss: 0.9601 - val_accuracy: 0.7499
Epoch 7/20
931/931 - 51s - loss: 0.2024 - accuracy: 0.9322 - val_loss: 0.9715 - val_accuracy: 0.7472
Epoch 8/20
931/931 - 51s - loss: 0.1880 - accuracy: 0.9376 - val_loss: 0.9867 - val_accuracy: 0.7480
Epoch 9/20
931/931 - 52s - loss: 0.1739 - accuracy: 0.9402 - val_loss: 1.0527 - val_accuracy: 0.7462
Epoch 10/20
931/931 - 53s - loss: 0.1635 - accuracy: 0.9437 - val_loss: 1.1078 - val_accura

In [19]:
 #save the model
print("Save Model")

joblib.dump(model, 'data/model_master.pkl', compress=1)
print("Create Model.. success")

Save Model


TypeError: cannot pickle 'tensorflow.python._tf_stack.StackSummary' object